In [ ]:
% run 0-utils.ipynb

In [ ]:
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
import scipy.sparse.csgraph
from tensorboardX import SummaryWriter
import skimage.transform as transform
import networkx as nx
from scipy import spatial
from gcnn.graph import knn,grid_coordinates
from gcnn.coarsening import graclus

In [ ]:
#from pygsp.reduction import graph_sparsify#, kron_reduction
from pygsp import graphs, filters, utils
from scipy.sparse.linalg import eigsh
from scipy.sparse import linalg
from scipy import sparse, stats
from scipy.sparse.csgraph import minimum_spanning_tree, dijkstra

In [ ]:
from pyamg import smoothed_aggregation_solver

In [ ]:
from gcnn.datasets import load_icebergs
icebergs = load_icebergs('train')

In [ ]:
icebergs.head()

In [ ]:
test = transform.rescale(icebergs.band_1.iloc[100].reshape(75, 75), 0.5, mode='constant')
plt.imshow(test);

In [ ]:
plt.spy(knn(grid_coordinates(3), k=1, metric='cityblock'));

In [ ]:
test_dist = spatial.distance.squareform(spatial.distance.pdist(test.reshape(-1, 1), metric='euclidean'))
plt.matshow(test_dist);

In [ ]:
test_dist[np.where(knn(grid_coordinates(38), k=2) == 0)] = 0

plt.subplot(121)
plt.imshow(test_dist[:100, :100])

plt.subplot(122)
plt.spy(test_dist);

In [ ]:
def calc_weight(n, root, pred, local_tree):
    
    if n==root:
        return 0, root, root
    
    parent = pred[root, n]

    w_p = local_tree[parent, n]
    
    gparent = pred[root, parent]
    
    if gparent != -9999:
        w_d = local_tree[gparent, parent]
    else:
        w_d = 1
        gparent = n
        
    w = 2./(1./w_p + 1./w_d)

    return w, parent, gparent

In [ ]:
plt.spy(test_dist)

In [ ]:
result, array = one_level_MST(test_dist)

In [ ]:
p = pd.DataFrame(array)
pd.DataFrame(p[0].value_counts()).sort_values(0, ascending=False)

In [ ]:
def one_level_MST(test_dist):

    test_dist_triu = np.triu(test_dist)
    Tree = minimum_spanning_tree(test_dist_triu)
    Tree = Tree + Tree.T
    local_tree = Tree.todense()

    distance_matrix, pred = dijkstra(Tree.todense(), directed=False, unweighted=True, return_predecessors=True)

    root = np.random.choice(np.arange(distance_matrix.shape[0]))
    even_nodes = distance_matrix[:, root] % 2 == 0
    even_nodes = np.arange(1444)[even_nodes]

    weight_tree = np.zeros((int(local_tree.shape[0]/2), int(local_tree.shape[1]/2)))
    
    array = [-1]*local_tree.shape[0]
    cluster = 0
    
    for n in even_nodes:
        new_weight, parent, gparent = calc_weight(n, root, pred, local_tree)
        array[n] = cluster
        array[parent] = cluster
        cluster += 1
        
        n = int(n/2)
        out_div = int(gparent/2)
        weight_tree[n, out_div] = new_weight
        weight_tree[out_div, n] = new_weight
        
    return weight_tree, array


In [ ]:
G = graphs.Graph(Tree)
G.set_coordinates()
G.plot()

In [ ]:
G2 = graphs.Graph(result)
G2.set_coordinates()
G2.plot()

In [ ]:
plt.spy(weight_tree)

In [ ]:
G1 = graphs.Graph(weight_tree)
G1.set_coordinates()
G1.plot()

In [ ]:
G = sp.sparse.csr.csr_matrix(knn(grid_coordinates(4), 1, metric='cityblock'))

In [ ]:
plt.spy(G.todense())

In [ ]:
graph, perm = graclus.coarsen(G, levels=2, self_connections=False)

In [ ]:
perm

In [ ]:
for g in graph:
    plt.subplot(121)
    plt.spy(g.todense())
    plt.subplot(122)
    nx.draw(nx.from_numpy_array(g.todense()))
    plt.show()

In [ ]:
gr = grid_coordinates(38)
gr.shape

In [ ]:
mask = knn(gr, k=3)
plt.spy(mask[:40, :40]);

In [ ]:
graph, perm = graclus.coarsen(sp.sparse.csr.csr_matrix(mask), levels=3, self_connections=False)